# Structuring reference and logic tree


d3.jsでツリーダイアグラムを書くのに適当なオブジェクトを構成するためのAPI

```
Node
 +- SelfNode                        Fact
 |   +- MyData       ---------------+     Model
 |   +- MyModel      -------------- | ----+
 |   +- MyInference  ------+        |     |
 |   +- MyPremise    ------+        |     | 
 +- RefNode                |        |     |
     +- RefData      ----- | -------+     |
     +- Theory       ----- | -------+     |
     +- RefModel     ----- | -------------+
     +- RefInference ------+
     +- RefPremise   ------+
                           Inference
```

child nodeとの関係  
改行
```
Fact:
    (nothing)

Model:
	assuming(premises)
	basedOn(theorum)
	composedOf(models)
	appliedTo(facts)

Inference:
	becauseOf(nodes)
	consistentWith(facts)
	supportedBy(nodes)
	estimatedBy(models)
	exceptingFor(nodes)
	indicatedBy(nodes)
	assuming(premises)
	extendingOn(references)
```

- SelfNodeはRefNodeの子になることができない．
- MyDataノード及びRefDataノードは子を持たない．

```
Node : {
    type : inference | data | theory | premise | model,
    title : String,
    content : Content,
    author : String,
    parent : String,
    edge : String,
    children : [Node]
    
}
```


In [1]:
var fs = require("fs")

var nodes = require("./js/logicNode.js")

var Node = require("./js/node.js");
var  MyModel = nodes.MyModel,
      MyData = nodes.MyData,
      MyInference = nodes.MyInference,
      MyPremise = nodes.MyPremise,
      RefData = nodes.RefData,
      RefInference = nodes.RefInference,
      RefModel = nodes.RefModel,
      Premise = nodes.Premise,
      Theory = nodes.Theory;


undefined

In [2]:
Node

[Function: Node]

In [3]:
console.log(Node.id)

undefined


undefined

## Opx zoning profile インバージョンモデルの例

In [4]:


// データ
var inversionDatas = Node.of(MyData, [
    {title:"start melt in fractionation",
     contents:"ホスト溶岩の全岩組成"}
])

inversionDatas.push(new MyModel({title:"opx integrated profile",
                                 contents:"もっとも初期からのイベントを記録したopxのFe/Mg, Cr2O3プロファイル"})
                               .assuming(Node.of(MyPremise,[
                                         {title:"異なる粒子のプロファイル対比",
                                         contents:"セグメントごとに線形に伸縮して重ね合わせられる"},
                                         {title:"粒子ごとの歴史",
                                         contents:"全ての粒子は注目している系の中に取り込まれてからずっと中にあった"}
                                        ]))
                    )
    

// 結晶成長モデルの仮定

var opxGrowthPremises = Node.of(MyPremise, [
    {title:"Constant stoichiometry",
    content:"結晶化のストイキオメトリーは一定"},
    {title:"Local equilibrium",
     contents:"界面で固相と液相の間での局所平衡が成立"},
    {title:"Constant partitioning",
     contents:"分配係数一定"},
    {title:"Liquid line of descent",
     contents:"各ステージのLLDは溶岩の全岩組成を通る"}
]);

var crystalGeometry=Node.of(MyPremise,[
    {title:"Crystal geometry",
     contents:"球対称，再溶融なし"},
    {title:"Homogenous growth",
     contents:"固相はそれぞれ液相中にただ一つ存在する結晶で代表される"}
])


// 拡散モデル
var opxDiffusionPremises = []
opxDiffusionPremises[0]=new MyPremise()
    .setTitle("Net time scale of diffusion")
    .setContents("成分によらずτtotalは共通")


var theoryOfInversion = Node.of(Theory,[
    {
    author:"Metropolice 1956",
    title:"MCMC method",
    contents:"MCMC and Metropolice's algorithm"
    },
    {
        author:"Fukushima & Nemoto, 1998",
        title:"PT-MCMC",
        contents:"Replica exchange"
    }  
])

// サブモデル
var opxGrowthModel = new MyModel()
    .setTitle("opx growth model")
    .setContents("Opx成長時の組成と成長量を対応させるモデル")
    .assuming(opxGrowthPremises)
    .assuming(crystalGeometry)

var diffusionModel = new MyModel()
    .setTitle("lattice diffusion model")
    .setContents("圧縮時間を導入した元素拡散モデル")
    .assuming(opxDiffusionPremises)
    .assuming(crystalGeometry)

var inversioningProfile = new MyModel()
    .setTitle("inversion model of diffused profile")
    .setContents("MCMCを用いて拡散の影響を受ける前の鉱物組成プロファイルを復元するモデル")
    .basedOn(theoryOfInversion)
    .assuming(Node.of(MyPremise,[
        {title:"データの不確かさ",
        contensts:"Integrated profileの移動分散で尤度を計算"}
    ]))
    .composedOf([opxGrowthModel,diffusionModel])
    .appliedTo(inversionDatas)

var restoredProfileOfOpx = new MyInference()
    .setTitle("Restored profile of opx")
    .setContents("MCMCサンプルの期待値から復元されたopx組成プロファイル")
    .estimatedBy(inversioningProfile)

//console.log(Node.getNodesFrom(restoredProfileOfOpx))
//console.log(Node.getLinksFrom(JSON.parse(Node.stringify(restoredProfileOfOpx))))

MyPremise {
  custom: LogicProps { author: 'self', bibtex: '' },
  children: [],
  contents: '成分によらずτtotalは共通',
  edges: [ 'assuming' ],
  parents: [ [Function] ],
  title: 'Net time scale of diffusion',
  type: 'premise' }

## 複数相飽和実験

In [5]:
var primaryMeltComposition = new MyInference({
    title:"HMAの初生メルト組成",
    content:"Table"
}).indicatedBy(restoredProfileOfOpx)

var alphaMELTS = new RefModel({
    author:"Asimow et al",
    title:"alphaMELTS",
    contents:"メルトモデルMELTSのCUIフロントエンド"
})

var multipleSaturation =new  MyModel({
    title: "Multiple saturation by MELTS",
    contents:"alphaMELTSによる複数相飽和実験モデル"
    })
    .basedOn(alphaMELTS)
    .appliedTo(primaryMeltComposition)
    
var restite = new MyInference({
    title:"mantle residue",
    contents:"Harzburgitic"
})
    .indicatedBy(multipleSaturation)
    .consistentWith(Node.of(MyData,[
        {
            title:"Olivine opx aggregate",
            contents:"olivineとメルトの反応組織"
        }
    ]))

undefined

In [6]:


var claim1 = new MyInference()
    .setTitle("HMAの初生マグマ")
    .setContents("これまで見積もられたよりも高MgO")
    .indicatedBy(primaryMeltComposition)

var claim2 = new MyInference()
    .setTitle("Final equilibrium condition")
    .setContents("これまでよりも高温，低圧,Harzburgite residue")
    .indicatedBy([multipleSaturation,restite])

var importantReport = [];
importantReport[0] = new RefInference({author:"Mashima"})
    .setTitle("SVBのテクトニクスとHMA形成条件")
    .setContents("横ずれ断層の運動による地殻の薄化と低圧での溶融")

importantReport[1] = new RefInference({author:"浅見ほか"})
    .setTitle("下部地殻物質がSVBの火成活動に関与")
    .setContents("felsicマグマは下部地殻の溶融メルト，HMAとも部分的に混合")
    .supportedBy(Node.of(RefData,[
        {
            author:"浅見ほか",
            title:"Xenocryst of granulite in HMA",
            contents:"下部地殻由来と思われる捕獲岩の存在"
        },
        {
            author:"浅見ほか",
            title:"Sr-Nd Isotope",
            contents:"HMAの一部やfelsic岩の同位体トレンドがgranulite質捕獲岩の方向に伸びる"
        }
    ]))





var conclusion = new MyInference({
    title:"Conclusion",
    contents:"SVBのHMAはマントルが浅所まで上昇し，部分溶融してできた．上昇マントルやHMAマグマの熱で下部地殻が溶融し，felsicなマグマが形成された．"
    })
    .estimatedBy([claim1,claim2])
    .consistentWith(importantReport)

//console.log(Node.getTreeFrom(conclusion)) // アップデートは自動的に反映される

claim1.setContents("Updated")
console.log(JSON.stringify(Node.getNodesFrom(conclusion),null,2))
console.log(Node.getLinksFrom(conclusion))
//console.log(Node.getNodesFrom(conclusion))
//console.log(Node.stringify(conclusion))

[
  {
    "custom": {
      "author": "self",
      "bibtex": ""
    },
    "contents": "SVBのHMAはマントルが浅所まで上昇し，部分溶融してできた．上昇マントルやHMAマグマの熱で下部地殻が溶融し，felsicなマグマが形成された．",
    "edges": [],
    "parents": [],
    "title": "Conclusion",
    "type": "inference",
    "depth": 0,
    "id": 0,
    "children": [
      {
        "custom": {
          "author": "self",
          "bibtex": ""
        },
        "contents": "Updated",
        "edges": [
          "estimated by"
        ],
        "parents": [
          0
        ],
        "title": "HMAの初生マグマ",
        "type": "inference",
        "depth": 1,
        "id": 1,
        "children": [
          {
            "custom": {
              "author": "self",
              "bibtex": ""
            },
            "contents": "",
            "edges": [
              "applied to",
              "indicated by"
            ],
            "parents": [
              24,
              1
            ],
            "title": "HMAの初生メルト組成",
            "type":

[ { source: 1, target: 0, edge: 'estimated by' },
  { source: 5, target: 24, edge: 'applied to' },
  { source: 5, target: 1, edge: 'indicated by' },
  { source: 6, target: 5, edge: 'indicated by' },
  { source: 7, target: 6, edge: 'estimated by' },
  { source: 8, target: 7, edge: 'based on' },
  { source: 9, target: 7, edge: 'based on' },
  { source: 10, target: 7, edge: 'assuming' },
  { source: 11, target: 7, edge: 'composed of' },
  { source: 15, target: 11, edge: 'assuming' },
  { source: 16, target: 11, edge: 'assuming' },
  { source: 17, target: 11, edge: 'assuming' },
  { source: 18, target: 11, edge: 'assuming' },
  { source: 19, target: 11, edge: 'assuming' },
  { source: 19, target: 12, edge: 'assuming' },
  { source: 20, target: 11, edge: 'assuming' },
  { source: 20, target: 12, edge: 'assuming' },
  { source: 12, target: 7, edge: 'composed of' },
  { source: 21, target: 12, edge: 'assuming' },
  { source: 13, target: 7, edge: 'applied to' },
  { source: 14, target: 7, edge

undefined

In [7]:
console.log(conclusion.toJson("width"))

fs.writeFileSync("./data/logic.json",conclusion.toJson("width"))

{
  "custom": {
    "author": "self",
    "bibtex": ""
  },
  "contents": "SVBのHMAはマントルが浅所まで上昇し，部分溶融してできた．上昇マントルやHMAマグマの熱で下部地殻が溶融し，felsicなマグマが形成された．",
  "edges": [],
  "parents": [],
  "title": "Conclusion",
  "type": "inference",
  "depth": 0,
  "id": 0,
  "children": [
    {
      "custom": {
        "author": "self",
        "bibtex": ""
      },
      "contents": "Updated",
      "edges": [
        "estimated by"
      ],
      "parents": [
        0
      ],
      "title": "HMAの初生マグマ",
      "type": "inference",
      "depth": 1,
      "id": 1,
      "children": [
        {
          "custom": {
            "author": "self",
            "bibtex": ""
          },
          "contents": "",
          "edges": [
            "applied to",
            "indicated by"
          ],
          "parents": [
            24,
            1
          ],
          "title": "HMAの初生メルト組成",
          "type": "inference",
          "depth": 1,
          "id": 5,
          "children": [
            {
 

undefined

In [6]:
class Contents{
    
}

class Figure extends Contents{
    
}

class Table extends Contents{
    
}

class Sentense extends Contents{
    
}

class SuperLink extends Contents{
    
}

ReferenceError: Ref is not defined

# Content class

- Sentences
- Table
- Figure

In [6]:
var a=[1,2,3,3,2]
a.filter(function (x, i, self) {
            return self.indexOf(x) === i;
        });

[ 1, 2, 3 ]

In [5]:
class Super{
    constructor(name){
        this.name=name
        this.type = "Super"
    }
    
    static batch(name){
        return new this(name)
    }
    
    static of(Class,list){
        return list.map(a=>new Class(a))
    }
    
}

class Sub extends Super{
    constructor(name){
        super(name)
        this.type="Sub"
    }
    
}



[Function: Sub]

In [10]:
var ins = ["a","b","c"].map(a=>Super.batch(a))
                        
console.log(ins)
console.log(ins.map(a=>a.type))

[ Super { name: 'a', type: 'Super' },
  Super { name: 'b', type: 'Super' },
  Super { name: 'c', type: 'Super' } ]
[ 'Super', 'Super', 'Super' ]


undefined

In [20]:
var ins = Sub.of(Sub,["a","b","c"])
                        
console.log(ins)
console.log(ins.map(a=>a.type))

c="k"

c instanceof Object && c.hasOwnProperty("author")

[ Sub { name: 'a', type: 'Sub' },
  Sub { name: 'b', type: 'Sub' },
  Sub { name: 'c', type: 'Sub' } ]
[ 'Sub', 'Sub', 'Sub' ]


false